In [9]:
# iš originalių Zive įrašų sukuriami įrašai *.npy ir anotacijų failai *.json.

# Planas:
# sukuriame tuščią dataframe df_list_selected, iš kurio darysime naują list.json
# Nuskaitome iš bendros talpyklos list.json
# pasidarom sąrašą userId:recordId1, recordId2,.... 
# ciklas per visus pacientus userId:
#   suformuojame sąrašą recordId: ekstrasistolių skaičius ir surušiuojame pagal ekstrasistolių
#   skaičių mažėjančia tvarka
#   Surikiavus įrašus, įrašus atrenkame tokiu būdu:
# 		a. Atrenkame pirmiausiai tuos įrašus, kurie turi daugiausiai ekstrasistolių. 
#       Jei visi įrašai turi ekstrasistolių, tai atrenkame pirmus 10 (iš surikiuotų įrašų pagal ekstrasistolių kiekį).
#       Jei ne visi įrašai turi ekstrasistoles, tai atrenkame, tik tuos įrašus, kurie turi ekstrasistoles
#       (bet ne daugiau 10 įrašų), ir jei įrašų atrinkta ne daugiau 9, pridedame dar vieną įrašą be ekstrasistolių.
# 		b. Jei pacientas ekstrasistolių neturi, tai atrenkame ne daugiau vieno (pirmo) to paciento įrašo

#   Atrinkus įrašus, papildome df_list_selected  
# ciklo per pacientus pabaiga
# įrašome visus atrinkus įrašus į record_selected
# įrašome df_list_selected į list.json.
# išvedame į ekraną suvestinę ir df_list_selected

# SubjCode = userNr + recordingNr 

import os, shutil, sys
import fnmatch
from icecream import ic
from pathlib import Path
import pandas as pd
import numpy as np

import json, time
from zive_util import zive_read_file_1ch, create_dir, create_SubjCode
from sklearn.model_selection import train_test_split


def get_rec_Id(rec_dir, userNr, recordingNr):
    # Patikriname, ar df_transl egzistuoja. 
    file = Path(rec_dir, 'df_transl.csv')
    if (file.exists()):
        # Nuskaitome vardų žodyną iš rec_dir aplanko
        file_path = Path(rec_dir, 'df_transl.csv')
        df_transl = pd.read_csv(file_path, index_col=0)
#       print(df_transl) 
         # Panaudodami df masyvą df_transl su įrašų numeriais iš įrašų eilės numerių gauname ZIVE numerius
        row = df_transl.loc[(df_transl['userNr'] == userNr) & (df_transl['recordingNr'] == recordingNr)]
        if row.empty:
            print("Klaida!")
            return None, None
        else:
            return row['userId'].values[0], row['recordingId'].values[0]
    else:
        print("df_transl neegzistuoja")
    
def get_rec_Nr(rec_dir, userId, recordingId):
    # Panaudodami df masyvą df_transl su įrašų numeriais iš ZIVE numerių gauname įrašų eilės numerius
    # Jei paciento Nr nėra - užvedamas įrašas
    
    # Patikriname, ar df_transl egzistuoja. Jei ne, sukuriame ir įrašome pirmą įraša
    file_path = Path(rec_dir, 'df_transl.csv')
    if (not file_path.exists()):
        # Paruošiame masyvą - žodyną numerių vertimui iš userId, registrationId į userNr, registrationNr ir atgal
        # ir įrašome į diską
        first_rec = {'userId':[userId], 'recordingId':[recordingId], 'userNr':[1000], 'recordingNr':[0]}
        df_transl = pd.DataFrame(first_rec)
        file_path = Path(rec_dir, 'df_transl.csv')
        df_transl.to_csv(file_path)
        # print(df_transl)
        return df_transl.loc[0, 'userNr'], df_transl.loc[0, 'recordingNr']

    # Jei egzistuoja, nuskaitome vardų žodyną iš rec_dir aplanko
    file_path = Path(rec_dir, 'df_transl.csv')
    df_transl = pd.read_csv(file_path, index_col=0)
    # print(df_transl)
    # Ieškome, ar yra įrašas su userId
    # Jei userId nerandame, sukuriame naują įrašą su userId, recordingId, userNr, recordingNr 
    if (df_transl.loc[(df_transl['userId'] == userId)]).empty:
        userNr = df_transl.loc[len(df_transl)-1, 'userNr'] + 1
        # print(f"{userNr=}")
        new_row = {'userId':userId, 'recordingId':recordingId, 'userNr':userNr, 'recordingNr':0}
        df_transl = df_transl.append(new_row, ignore_index=True)
        file_path = Path(rec_dir, 'df_transl.csv')
        df_transl.to_csv(file_path)
        return userNr, 0
    
    # Jei radus userId, randame kad šio paciento įrašo recordingId jau yra, su įrašais nieko nedarome
    row = df_transl.loc[(df_transl['userId'] == userId) & (df_transl['recordingId'] == recordingId)]
    if not row.empty:
        # print(row)
        return row['userNr'].values[0], row['recordingNr'].values[0]
    else:
    # Jei šio paciento įrašo su recordingId nėra, formuojame įrašą
        rows = df_transl.loc[(df_transl['userId'] == userId)]
        recordingNr = max(rows['recordingNr'].to_list()) + 1
        userNr = rows['userNr'].values[0]
        new_row = {'userId':userId, 'recordingId':recordingId, 'userNr':userNr, 'recordingNr':recordingNr}
        df_transl = df_transl.append(new_row, ignore_index=True)
        file_path = Path(rec_dir, 'df_transl.csv')
        df_transl.to_csv(file_path)
        return userNr, recordingNr    
    


my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'ZIVE_BUFFER'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

rec_dir = Path(db_path,'records_selected')

# Nuoroda į aplanką su transformuotu EKG duomenų rinkiniu
rec_dir_npy = Path(db_path, 'records_npy')

# Paliekamų anotacijų sąrašas
annot_list = ['N','S','V','U']

# Diskretizavimo dažnis
fs_zive = 200

print("\nSkriptas zive įrašų transformacijai\n")

print("Išeities duomenys skaitomi iš:", rec_dir)
print("Transformuoti duomenys rašomi į:", rec_dir_npy)
print("\n")


OS in my system :  win32

Skriptas zive įrašų transformacijai

Išeities duomenys skaitomi iš: C:\DI\Data\MIT&ZIVE\ZIVE_BUFFER\records_selected
Transformuoti duomenys rašomi į: C:\DI\Data\MIT&ZIVE\ZIVE_BUFFER\records_npy




In [10]:

# Sukūriame aplanką EKG sekų įrašymui
create_dir(rec_dir_npy)

pd.set_option("display.max_rows", 1000, "display.max_columns", 20)
pd.set_option('display.width', 2000)

# Nuskaitome Zive įrašų talpykloje laikomų įrašų sąrašą
file_path = Path(rec_dir, 'list.json')
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())

df_list = pd.json_normalize(data, record_path =['data'])

# print(df_list)
# Sugrupuojame įrašus į to paties paciento grupes
grouped = df_list.groupby(['userId','recordingId'])
print(grouped)
print("\n")

SubjCodes = []

# Perrašome įrašus nauju formatu į kitą folderį
for key in grouped.groups:
    userId = key[0]
    recordingId = key[1]
    
    print(userId, recordingId)
    userNr, recordingNr = get_rec_Nr(rec_dir_npy, userId, recordingId)
    SubjCode = create_SubjCode(userNr, recordingNr)  
    print("SubjCode: ", SubjCode, userNr, recordingNr)

    SubjCodes.append(SubjCode)

    file_path = Path(rec_dir, recordingId)
    signal = zive_read_file_1ch(file_path)  

    file_path = Path(rec_dir_npy, str(SubjCode) + '.npy')
    with open(file_path, 'wb') as f:
        np.save(f, signal)

    src = Path(rec_dir, recordingId + '.json')
    dst = Path(rec_dir_npy, str(SubjCode) + '.json')
    shutil.copy2(src, dst)

info = {
    'db_folder':db_folder,
    'fs': fs_zive,
    'SubjCodes':SubjCodes,
    'annot_list':annot_list
    }

file_name = Path(rec_dir_npy,'info_create_z.json')
with open(file_name, 'w') as f:
    json.dump(info, f)
    
print("\nSchemos parametrai įrašyti į failą: ", file_name, "\n" )

Directory 'C:\DI\Data\MIT&ZIVE\ZIVE_BUFFER\records_npy' already exists


6034c808d6c2740008035ede 607ee16394e6ea7ee11ab93b
SubjCode:  10000 1000 0
6034c808d6c2740008035ede 60f945e187cf66e97e37810e
SubjCode:  10001 1000 1
6034c808d6c2740008035ede 6133101f3d08d40385cdc8ce
SubjCode:  10002 1000 2
6034c808d6c2740008035ede 615516b62312e50c3f168e38
SubjCode:  10003 1000 3
607efbd094e6ea3b5a1ab959 60806e0e94e6ea62191ab966
SubjCode:  10010 1001 0
60a917b354352a3df86dc1f2 60a92aa454352a03116dc1f5
SubjCode:  10020 1002 0
60a917b354352a3df86dc1f2 60efffba87cf660aac377fe7
SubjCode:  10021 1002 1
60a917b354352a3df86dc1f2 60f9188487cf66240e3780bf
SubjCode:  10022 1002 2
60a917b354352a3df86dc1f2 60f9188487cf6632713780c5
SubjCode:  10023 1002 3
60a917b354352a3df86dc1f2 60f9188487cf6678b43780c3
SubjCode:  10024 1002 4
60a917b354352a3df86dc1f2 60f9188487cf669c7e3780b8
SubjCode:  10025 1002 5
60e1d80f93b55b41529e9eaa 60e1db5a93b55b14359e9ec3
SubjCode:  10030 1003 0
60e1d80f93b55b41529e9eaa 60e1db5a93b55b

In [11]:
file_path = Path(rec_dir_npy, 'df_transl.csv')
df_transl = pd.read_csv(file_path, index_col=0)
print(df_transl)

# recNrs = df_transl["recordingNr"].to_list()
# print(recNrs)


userId = '6034c808d6c2740008035ede'
rows = df_transl.loc[(df_transl['userId'] == userId)]
print(rows)

userNr = rows['userNr'].values[0]
print(userNr)
max_recordingNr = max(rows['recordingNr'].to_list())
print("max_recordingNr=",max_recordingNr)


                       userId               recordingId  userNr  recordingNr
0    6034c808d6c2740008035ede  607ee16394e6ea7ee11ab93b    1000            0
1    6034c808d6c2740008035ede  60f945e187cf66e97e37810e    1000            1
2    6034c808d6c2740008035ede  6133101f3d08d40385cdc8ce    1000            2
3    6034c808d6c2740008035ede  615516b62312e50c3f168e38    1000            3
4    607efbd094e6ea3b5a1ab959  60806e0e94e6ea62191ab966    1001            0
5    60a917b354352a3df86dc1f2  60a92aa454352a03116dc1f5    1002            0
6    60a917b354352a3df86dc1f2  60efffba87cf660aac377fe7    1002            1
7    60a917b354352a3df86dc1f2  60f9188487cf66240e3780bf    1002            2
8    60a917b354352a3df86dc1f2  60f9188487cf6632713780c5    1002            3
9    60a917b354352a3df86dc1f2  60f9188487cf6678b43780c3    1002            4
10   60a917b354352a3df86dc1f2  60f9188487cf669c7e3780b8    1002            5
11   60e1d80f93b55b41529e9eaa  60e1db5a93b55b14359e9ec3    1003            0

In [12]:
# https://www.geeksforgeeks.org/python-shutil-copy2-method/#:~:text=shutil.copy
# 2%20%28%29%20method%20in%20Python%20is%20used%20to,destination%20can%20be%20a%20file%20or%20a%20directory.

# Susirandame anotacijų pasiskirstymą per pacientus ir pacientų skaičių
df_sum = df_list.groupby(['userId'],sort = False).sum()
# print(df_sum)
# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = df_list['userId'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(df_list)}')
print(f'\nĮrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
result.index.rename ('userID', inplace= True)
result.drop(labels=['nr','incl'], axis=1, inplace=True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
print(result)

# Įrašome failų sąrašą į diską
file_path = Path(rec_dir_npy,'list.json')       
df_list.to_json(file_path, orient = 'table', index=False)
print(f'\nFailų sąrašas įrašytas:  {file_path}')



Viso pacientų: 50  EKG įrašų: 289

Įrašų pasiskirstymas per pacientus
                              N    S     V  U  F1  F2  F3  count
userID                                                          
6034c808d6c2740008035ede  12225    7     0  0   0   0   0      4
607efbd094e6ea3b5a1ab959    835    0    30  0   0   0   0      1
60a917b354352a3df86dc1f2   9970   16    10  5   0   0   0      6
60e1d80f93b55b41529e9eaa   7238   11    72  0   0   0   0      7
613b1c6f3d08d4370acdc8f3    651    3     0  0   0   0   0      1
613b1d0c3d08d413ffcdc8f6   7930   26   145  0   0   0   0     10
613b1d673d08d4d1f3cdc8f8   7787   99     0  5   0   0   0     10
6143507abd0cc5051b275171   1885    5     4  0   0   0   0      3
6144c4fbbd0cc552e427535f   6080    0  2275  0   0   0   0     10
6144c532bd0cc54c83275360    787    0     0  0   0   0   0      1
6144c588bd0cc52ba0275362   5280    0   413  0   0   0   0     10
6144c5b1bd0cc5a681275363    903    0     0  0   0   0   0      1
6144c682bd0cc5acb72